In [3]:
import os
os.getcwd()
# os.chdir("/home/software/annoduf")

'/home/software/annoduf'

In [ ]:
import pandas as pd
df = pd.read_csv("new_duf/difference_v35_v37.csv")
df[df['Number_of_Sequence'] == 1].reset_index(drop=True).to_csv("new_duf/Codes/DUFs_with_1_Sequence.csv", index=False)
df[df['Number_of_Sequence'] != 1].reset_index(drop=True).to_csv("new_duf/Codes/DUFs_More_than_1_Sequence.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("new_duf/Codes/DUFs_More_than_1_Sequence.csv")
for i, split_df in enumerate(np.array_split(df, 3), start=1):
    split_df.reset_index(drop=True).to_csv(f"new_duf/Codes/DUFs_More_than_1_Sequence_{i}.csv", index=False)

**Extract Description from PFam**

In [6]:
import pandas as pd
dufANDpfamID = pd.read_csv("new_duf/Test_List_of_DUFs.tsv", sep="\t")
pfamLst = list(dufANDpfamID["pfamID"])
extract_lines = False
with open("new_duf/template_Pfam_A.full", 'r') as file:
    for line in file:
        if line.startswith("#=GF AC") and line.replace('#=GF AC','').strip() in pfamLst:
            accID = line.replace('#=GF AC','').strip()
            extract_lines = True
            gf_cc_lines = []
            continue
        elif extract_lines and line.startswith("#=GF CC"):
            gf_cc_lines.append(line.strip())
        elif line.startswith("#=GF SQ"):
            if extract_lines:
                if gf_cc_lines != []: 
                    gf_cc_lines =  " ".join([line.replace("#=GF CC", "").lstrip().strip() for line in gf_cc_lines])
                    dufANDpfamID.loc[dufANDpfamID['pfamID'] == accID, 'description'] = gf_cc_lines
                else: 
                    dufANDpfamID.loc[dufANDpfamID['pfamID'] == accID, 'description'] = ""
    
            extract_lines = False
dufANDpfamID.to_csv("new_duf/New_Test_List_of_DUFs.tsv", sep="\t")